In [ ]:
# https://mne.tools/stable/auto_examples/preprocessing/xdawn_denoising.html#sphx-glr-auto-examples-preprocessing-xdawn-denoising-py

In [1]:
import numpy as np
import pandas as pd

import mne
from mne import Epochs, pick_channels, find_events
from mne.io import read_raw_bdf

import matplotlib.pyplot as plt
%matplotlib qt

In [14]:
conds = ['1', '2', '3']
sides = ['Right', 'Left', 'ClassicMod']
perfs = ['good', 'bad']

report = prep_report(conds, sides, perfs)

In [5]:
#raw = read_raw_bdf("/Volumes/Cortex/PhD/Open Datasets/Data/AP140119.bdf", preload=True)
raw = read_raw_bdf("/Users/nick/Documents/PhD/CAE - InLook/CAE - NeuroTracker/F12F2/EEG/NT All.bdf", preload=True)

raw = raw.drop_channels(ch_names=['EXG1','EXG2','EXG3','EXG6','EXG7'])
raw = raw.set_eeg_reference(ref_channels=['EXG4', 'EXG5'])
#raw = raw.set_channel_types({'EXG4': 'misc', 'EXG5': 'misc'})

# Filter
#raw = raw.filter(.1, 20., fir_design='firwin', skip_by_annotation='edge')

#Max Events in File...
#---------------------------
#32: 88
#1:  172
#2:  84
#4:  84
#8:  84
#16: 4
#---------------------------

# Triggers
# 1 - New Spheres.
# 2 - Red Spheres.
# 4 - Start Moving.
# 8 - Stop Moving.
# 16 - TBD
# 32 - TBD (juste juste avant le 1, so I guess fin de trial)

Extracting EDF parameters from /Users/nick/Documents/PhD/CAE - InLook/CAE - NeuroTracker/F12F2/EEG/NT All.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5986303  =      0.000 ...  2923.000 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 67585 samples (33.000 sec)



In [7]:
events = find_events(raw)
events

Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]


array([[ 392594,       0,       1],
       [ 393350,       0,       1],
       [ 403553,       0,       2],
       ...,
       [5942198,       0,       8],
       [5948002,       0,      32],
       [5948624,       0,       1]])

In [8]:
import collections

total = 0

frequency = collections.Counter(events[:,2])
events_frequency = dict(frequency)

print('Max Events in File...')
print("---------------------------")
for e in set(events[:,2]):
    print("{}: \t\t{}".format(e, frequency[e]))
print("---------------------------")

Max Events in File...
---------------------------
32: 		88
1: 		172
2: 		84
4: 		84
8: 		84
16: 		4
---------------------------


In [9]:
x = raw.plot(events=events, block=True, lowpass=40)

Setting up low-pass filter at 40 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 40.00 Hz: -6.02 dB

Using matplotlib as 2D backend.
Channels marked as bad: none


KeyboardInterrupt: 

In [15]:
df_behav = pd.read_csv('/Users/nick/Documents/PhD/CAE - InLook/CAE - NeuroTracker/F12F2/Behav/F12F2/F12F2_2021-11-12 15.00.26.csv', sep=';')
df_behav

,#Trial,Synch,NbTargets,NbDistractors,Speed,Mode,Wallsize,Results,[TID_Entered_1],[TID_Entered_2],...,MiddleTimeCross.3,Ball Id.4,TimeCross,Ball Id.5,TimeCross.1,Ball Id.6,TimeCross.2,Ball Id.7,TimeCross.3,Unnamed: 36
0,1,True,3,5,1.000000,ClassicMod,10,0,1,2,...,--,--,--,--,--,--,--,--,--,NaN
1,2,False,3,5,0.630957,ClassicMod,10,3,2,3,...,--,--,--,--,--,--,--,--,--,NaN
2,3,False,3,5,1.000000,ClassicMod,10,2,7,8,...,--,--,--,--,--,--,--,--,--,NaN
3,4,False,3,5,0.630957,ClassicMod,10,3,1,4,...,--,--,--,--,--,--,--,--,--,NaN
4,5,False,3,5,0.794328,ClassicMod,10,3,3,7,...,--,--,--,--,--,--,--,--,--,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,17,False,1,7,1.010000,Left,10,0,2,0,...,--,--,--,--,--,--,--,--,--,NaN
101,18,False,2,6,1.010000,Left,10,2,4,6,...,--,--,--,--,--,--,--,--,--,NaN
102,19,False,1,7,1.010000,Left,10,1,4,0,...,--,--,--,--,--,--,--,--,--,NaN
103,20,True,3,5,1.010000,Left,10,2,5,6,...,--,--,--,--,--,--,--,--,--,NaN


In [16]:
# Drop 21 first (Practice Session)
df_behav.drop(index=df_behav.index[:21], axis=0, inplace=True)
df_behav

,#Trial,Synch,NbTargets,NbDistractors,Speed,Mode,Wallsize,Results,[TID_Entered_1],[TID_Entered_2],...,MiddleTimeCross.3,Ball Id.4,TimeCross,Ball Id.5,TimeCross.1,Ball Id.6,TimeCross.2,Ball Id.7,TimeCross.3,Unnamed: 36
21,1,True,2,6,1.41,Left,10,2,4,6,...,--,--,--,--,--,--,--,--,--,NaN
22,2,False,3,5,1.41,Right,10,2,4,7,...,--,--,--,--,--,--,--,--,--,NaN
23,3,False,1,7,1.41,Right,10,1,8,0,...,--,--,--,--,--,--,--,--,--,NaN
24,4,False,2,6,1.41,Left,10,1,3,7,...,--,--,--,--,--,--,--,--,--,NaN
25,5,False,1,7,1.41,ClassicMod,10,0,2,0,...,--,--,--,--,--,--,--,--,--,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,17,False,1,7,1.01,Left,10,0,2,0,...,--,--,--,--,--,--,--,--,--,NaN
101,18,False,2,6,1.01,Left,10,2,4,6,...,--,--,--,--,--,--,--,--,--,NaN
102,19,False,1,7,1.01,Left,10,1,4,0,...,--,--,--,--,--,--,--,--,--,NaN
103,20,True,3,5,1.01,Left,10,2,5,6,...,--,--,--,--,--,--,--,--,--,NaN


In [17]:
events_dict = dict()
for cond in conds:
    events_dict[cond] = dict()
    for side in sides:
        events_dict[cond][side] = dict()
        for perf in perfs:
            events_dict[cond][side][perf] = []

nb_trials = 0
for e in events:
    cur_cond = None
    cur_side = None
    cur_perf = None
    if e[2] == 2:
        cur_cond = str(df_behav['NbTargets'].values[nb_trials])
        cur_side = str(df_behav['Mode'].values[nb_trials])
        cur_perf = 'good' if (int(df_behav['NbTargets'].values[nb_trials]) == int(df_behav['Results'].values[nb_trials])) else 'bad'
        if cur_cond is not None and cur_side is not None and cur_perf is not None:
            events_dict[cur_cond][cur_side][cur_perf].append(e)
        
        nb_trials = nb_trials + 1
    
report = fill_report(report, 'F12F2', events_dict)
report
#print('Right: {} | Left: {} | Classic: {}'.format(len(events_dict['Right']), len(events_dict['Left']), len(events_dict['ClassicMod'])))

,1,2,3,Right,Left,ClassicMod,good,bad,1-Right,1-Right-good,...,3-Right-bad,3-bad,3-Left,3-Left-good,3-Left-bad,3-ClassicMod,3-ClassicMod-good,3-ClassicMod-bad,checksum,notes
F12F2,28,28,28,30,30,24,53,31,10,10,...,7,12,10,6,4,8,7,1,0,


In [18]:
epoch_length = 10
epoch_tmin = -0.2
baseline_corr = (-0.2, 0)

epochs_dict = dict()

epochs_dict['Right'] = Epochs(raw, events_dict['3']['Right']['good'], tmin=epoch_tmin, tmax=epoch_length, baseline=baseline_corr, preload=True)
epochs_dict['Left'] = Epochs(raw, events_dict['3']['Left']['good'], tmin=epoch_tmin, tmax=epoch_length, baseline=baseline_corr, preload=True)

times = epochs_dict['Right'].times

#x = epochs_dict['Right'].plot()


Not setting metadata
Not setting metadata
3 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 3 events and 20891 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
6 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 6 events and 20891 original time points ...
0 bad epochs dropped


In [19]:
epochs_right = epochs_dict['Right'].copy().pick_channels(['PO8'])
epochs_left = epochs_dict['Right'].copy().pick_channels(['PO7'])

evoked_right = epochs_right.average().data
evoked_left = epochs_left.average().data

cda = evoked_right - evoked_left

plt.plot(times, cda[0])

In [20]:
epochs_right = epochs_dict['Left'].copy().pick_channels(['PO8'])
epochs_left = epochs_dict['Left'].copy().pick_channels(['PO7'])

evoked_right = epochs_right.average().data
evoked_left = epochs_left.average().data

cda = evoked_right - evoked_left

plt.plot(times, cda[0])

In [22]:
#epochs_dict['Right']
epochs_dict['Left']

Number of events,6
Events,2: 6
Time range,-0.200 – 10.000 sec
Baseline,-0.200 – 0.000 sec


# Report Functions

In [13]:
def prep_report(conds, sides, perfs):
    columns = []
    columns.extend(conds)
    columns.extend(sides)
    columns.extend(perfs)

    for cond in conds:
        for side in sides:
            columns.append('{}-{}'.format(cond, side))
            for perf in perfs:
                columns.append('{}-{}-{}'.format(cond, side, perf))
                if '{}-{}'.format(cond, perf) not in columns:
                    columns.append('{}-{}'.format(cond, perf))
    
    columns.append('checksum')
    columns.append('notes')
    
    return pd.DataFrame(columns=columns)


def fill_report(report, filename, specific_events):
    cur_row = list()
    cur_row.extend([0] * (len(report.keys()) - 1)) # Excluding notes columns
    cur_row.extend([''])
    report = report.append(pd.DataFrame([cur_row], columns=report.columns, index=[filename]))
    
    # Looking for level 1: conditions
    for cond in specific_events.keys():
        if cond not in report.keys():
            raise ValueError('Event Key ({}) not in Report Columns.'.format(k))
        
        for side in specific_events[cond].keys():
            if side not in report.keys():
                raise ValueError('Event Key ({}) not in Report Columns.'.format(k))
            
            for perf in specific_events[cond][side].keys():
                if perf not in report.keys():
                    raise ValueError('Event Key ({}) not in Report Columns.'.format(k))
                
                #print("Adding: {}-{}-{} ({}) to: {}".format(cond, side, perf, len(specific_events[cond][side][perf]), cond))
                report.at[filename, cond] = report.at[filename, cond] + len(specific_events[cond][side][perf])
                report.at[filename, side] = report.at[filename, side] + len(specific_events[cond][side][perf])
                report.at[filename, perf] = report.at[filename, perf] + len(specific_events[cond][side][perf])
                report.at[filename, '{}-{}'.format(cond, side)] = report.at[filename, '{}-{}'.format(cond, side)] + len(specific_events[cond][side][perf])
                report.at[filename, '{}-{}'.format(cond, perf)] = report.at[filename, '{}-{}'.format(cond, perf)] + len(specific_events[cond][side][perf])
                report.at[filename, '{}-{}-{}'.format(cond, side, perf)] = report.at[filename, '{}-{}-{}'.format(cond, side, perf)] + len(specific_events[cond][side][perf])
    
    return report


def get_report(report, conds, sides, perfs, other_cols):
    sub_report_cols = []
    for cond in conds: sub_report_cols.append(cond)
    for side in sides: sub_report_cols.append(side)
    for perf in perfs: sub_report_cols.append(perf)
    
    for cond in conds: 
        for side in sides: 
            sub_report_cols.append('{}-{}'.format(cond,side))
        
    for cond in conds: 
        for side in sides: 
            for perf in perfs: 
                sub_report_cols.append('{}-{}-{}'.format(cond, side, perf))
    
    return report[sub_report_cols]